In [ ]:
import pandas as pd

In [ ]:
wedstrijden_df = pd.read_excel("./Data/wedstrijden.xlsx")
cornerballen_df = pd.read_excel("./Data/cornerballen.xlsx")

In [ ]:
wedstrijden_df.head()

In [ ]:
wedstrijden_df.dtypes

In [ ]:
cornerballen_df.dtypes

In [ ]:
full_df = wedstrijden_df.merge(cornerballen_df, "left", "wedstrijd_id")

In [ ]:
full_df.head()

In [ ]:
test = full_df[["uitslag", "klassement_uitslag", "score_voor"]]

In [ ]:
### functie voor spanning van match/ positie in het klassement 
# voorlopig 0/1 goals verschil zeer spannend, 2/3 goals verschil spannend, anders niet spannend

def spannendheid_tijdens_match(score_kolom, sprong = 2):
    score_1, score_2 = map(int, score_kolom.split("v"))
    verschil = abs(score_1 - score_2)

    if verschil < sprong:
        return "zeer spannend"
    elif verschil < sprong*2:
        return "spannend"
    else:
        return "niet spannend"

In [ ]:
test["spanning_op_schepcorner"] = test["score_voor"].apply(lambda x: spannendheid_tijdens_match(x, 3))

In [ ]:
# spanning voor de match berekenen
def spannendheid_voor_wedstijd(klassement_kolom,thuisploeg_kolom, kern_kolom,  weight_concurrent = 0.8):
    positie_thuis, positie_uit = map(int, klassement_kolom.split("v"))
    
    # stress door rechtstreekse tegenstander
    verschil = abs(positie_thuis - positie_uit)
    if verschil < 3:
        rechtstreekse_tegenstander_score = 1
    elif verschil < 5: 
        rechtstreekse_tegenstander_score = 1/2
    else:
        rechtstreekse_tegenstander_score = 0
    
    # stress door positie in het klassement
    if thuisploeg_kolom == 1:
        positie = int(klassement_kolom.split("v")[0])
    else:
        positie = int(klassement_kolom.split("v")[1])

    klassement_spanning = 0
    if kern_kolom != 4:
        if positie in [1,2,3,12,13,14,15]:
            klassement_spanning = 1
    else:
        # geen degradatie op laagste niveau
        if positie in [1,2,3]:
            klassement_spanning = 1
    
    # bepaling van uiteindelijke score
    finale_spannenheid_voor_match_score = weight_concurrent * rechtstreekse_tegenstander_score + (1-weight_concurrent) * klassement_spanning
    return finale_spannenheid_voor_match_score


In [ ]:
spannendheid_voor_wedstijd("1v7", 1, 3, 0.8)

In [ ]:
test = full_df[["uitslag", "klassement_uitslag", "score_voor", "niveau", "is_thuisploeg"]]

In [ ]:
test["spannendheid_voor_match"] = test.apply(lambda x : spannendheid_voor_wedstijd(x["klassement_uitslag"], x["is_thuisploeg"], x["niveau"]), axis=1)

In [ ]:
test

### Impact van de schepcorner op matchuitslagen

In [ ]:
test = full_df[["wedstrijd_id", "uitslag", "goal", "is_thuisploeg"]]

In [ ]:
test

In [ ]:
# UItkomst van 1 schepcorner bepaald match



In [ ]:
data = {
    "wedstrijd_id": [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4],
    "uitslag": ["1v1", "1v1", "1v1", "11v2", "11v2", "11v2", "4v6", "4v6", "4v6", "4v6", "2v1", "2v1"],
    "goal": [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1],  
    "is_thuisploeg": [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0]  
}
df = pd.DataFrame(data)


In [ ]:
# strengste impact, uitkomst van 1 schepcorner in de match veranderen, is bepalend
def schepcorner_impact(df):
    results = []

    for match_id, match_data in df.groupby("wedstrijd_id"):
        uitslag_str = match_data["uitslag"].iloc[0]  # iloc selecteert eerste rij van match_data
        score_thuis, score_uit = map(int, uitslag_str.split("v"))  
        score_verschil = abs(score_thuis - score_uit)

        # Controleer of er een schepcorner is (indien alle is_thuisploeg en goal NA zijn, dan geen schepcorner)
        heeft_schepcorner = match_data["goal"].notna().any()

        if not heeft_schepcorner:
            results.append((match_id, False))  # Geen schepcorner? Geen impact.
            continue  

        # Gelijkspel: altijd impact als er een schepcorner was
        if score_thuis == score_uit:
            results.append((match_id, True))
            continue

        # Bepaal winnaar en verliezer
        if score_thuis > score_uit:
            winnaar = 1
            verliezer = 0
        else:
            winnaar = 0
            verliezer = 1

        # Controleer of winnaar minstens één schepcorner scoorde
        winnaar_heeft_gescoord = ((match_data["is_thuisploeg"] == winnaar) & (match_data["goal"] == 1)).any()

        # Controleer of verliezer minstens één schepcorner miste
        verliezer_heeft_gemist = ((match_data["is_thuisploeg"] == verliezer) & (match_data["goal"] == 0)).any()

        # Strikte voorwaarde: maximaal 1 verschil en schepcorner moet impact hebben
        if score_verschil == 1 and (winnaar_heeft_gescoord or verliezer_heeft_gemist):
            results.append((match_id, True))
        else:
            results.append((match_id, False))

    return pd.DataFrame(results, columns=["wedstrijd_id", "schepcorner_impact"])


In [ ]:

# test
impact_df = schepcorner_impact(df)
print(impact_df)

Minder strenge impact van de schepcorner. Heeft de wedstrijd nog dezelfde uitslag (zelfde winnaar, verlieze of nog steeds gelijkspel moesten er geen schepcorners zijn) of moesten alle schepcorners gemist worden

In [ ]:
def bepaal_winnaar_verliezer(uitslag_str):
    thuis_score, uit_score = map(int, uitslag_str.split('v'))
    
    if thuis_score > uit_score:
        return 1, 0  
    elif uit_score > thuis_score:
        return 0, 1  
    else:
        return None, None  

In [ ]:
# Functie om schepcorner-impact te bepalen
def impact_alle_schepcorners_gemist(uitslag_str, match_data):
    # Bepaal het aantal schepcorner-goals per ploeg
    schepcorner_goals_thuis = match_data[match_data["is_thuisploeg"] == 1]["goal"].sum()
    schepcorner_goals_uit = match_data[match_data["is_thuisploeg"] == 0]["goal"].sum()
    
    # Bereken nieuwe uitslag zonder schepcorners
    thuis_score, uit_score = map(int, uitslag_str.split('v'))
    thuis_score -= schepcorner_goals_thuis
    uit_score -= schepcorner_goals_uit

    # Bepaal de oorspronkelijke en nieuwe winnaar
    oorspronkelijke_winnaar = bepaal_winnaar_verliezer(uitslag_str)
    nieuwe_winnaar = bepaal_winnaar_verliezer(f"{thuis_score}v{uit_score}")
    
    # Schepcorner heeft impact als de winnaar verandert
    return oorspronkelijke_winnaar != nieuwe_winnaar


In [ ]:
impact_resultaten = df.groupby("wedstrijd_id").apply(
    lambda match_data: impact_alle_schepcorners_gemist(match_data["uitslag"].iloc[0], match_data)
)

# Voeg het resultaat toe aan het dataframe
df = df.merge(impact_resultaten.rename("schepcorner_impact"), on="wedstrijd_id")

# Print het resultaat
print(df[["wedstrijd_id", "schepcorner_impact"]].drop_duplicates())

In [ ]:
# impact schepcorner moest elke schepcorner gescoord worden
def impact_alle_schepcorners_gescoord(uitslag_str, match_data):
    # tel het aantal gemiste corners per team
    gemiste_schepcorners_thuis = ((match_data["is_thuisploeg"] == 1) & (match_data["goal"] == 0)).sum()
    gemiste_schepcorners_uit = ((match_data["is_thuisploeg"] == 0) & (match_data["goal"] == 0)).sum()

    # Bereken nieuwe uitslag als alle schepcorners gescoord waren
    thuis_score, uit_score = map(int, uitslag_str.split('v'))
    thuis_score += gemiste_schepcorners_thuis  # Voeg alleen gemiste schepcorners toe
    uit_score += gemiste_schepcorners_uit

    # Bepaal oorspronkelijke en nieuwe winnaar
    oorspronkelijke_winnaar = bepaal_winnaar_verliezer(uitslag_str)
    nieuwe_winnaar = bepaal_winnaar_verliezer(f"{thuis_score}v{uit_score}")

    # Schepcorner heeft impact als de winnaar verandert
    return oorspronkelijke_winnaar != nieuwe_winnaar


In [ ]:

# Pas de functie toe per wedstrijd_id
impact_resultaten_max = df.groupby("wedstrijd_id").apply(
    lambda match_data: impact_alle_schepcorners_gescoord(match_data["uitslag"].iloc[0], match_data)
)

# Voeg het resultaat toe aan het dataframe
df = df.merge(impact_resultaten_max.rename("schepcorner_impact_max"), on="wedstrijd_id")

# Print het resultaat
print(df[["wedstrijd_id", "schepcorner_impact_max"]].drop_duplicates())

# impact van de schepcorner als alle uitkommsten van schepcorners gewisseld kunnen worden, minst streng

In [ ]:
def heeft_schepcorner_impact_mild(uitslag_str, match_data):

    # Tel gescoorde en gemiste schepcorners per team
    gescoorde_schepcorners_thuis = ((match_data["is_thuisploeg"] == 1) & (match_data["goal"] == 1)).sum()
    gemiste_schepcorners_thuis = ((match_data["is_thuisploeg"] == 1) & (match_data["goal"] == 0)).sum()
    
    gescoorde_schepcorners_uit = ((match_data["is_thuisploeg"] == 0) & (match_data["goal"] == 1)).sum()
    gemiste_schepcorners_uit = ((match_data["is_thuisploeg"] == 0) & (match_data["goal"] == 0)).sum()
    
    # Huidige score bepalen
    thuis_score, uit_score = map(int, uitslag_str.split('v'))
    oorspronkelijke_winnaar = bepaal_winnaar_verliezer(uitslag_str)
    
    # Als het gelijkspel was en er waren schepcorners, dan heeft het impact
    if oorspronkelijke_winnaar is None and (gescoorde_schepcorners_thuis + gescoorde_schepcorners_uit + gemiste_schepcorners_thuis + gemiste_schepcorners_uit) > 0:
        return True
    
    # Bepaal de minimale score van de oorspronkelijke winnaar en de maximale score van de verliezer
    if oorspronkelijke_winnaar == 1:
        min_winnaar_score = thuis_score - gescoorde_schepcorners_thuis
        max_verliezer_score = uit_score + gemiste_schepcorners_uit
    else:
        min_winnaar_score = uit_score - gescoorde_schepcorners_uit
        max_verliezer_score = thuis_score + gemiste_schepcorners_thuis

    # Nieuwe uitslag bepalen zonder schepcorners
    nieuwe_winnaar = bepaal_winnaar_verliezer(f"{min_winnaar_score}v{max_verliezer_score}")
    
    # Impact als de winnaar verandert
    return oorspronkelijke_winnaar != nieuwe_winnaar


In [ ]:

# Pas de functie toe per wedstrijd_id
impact_resultaten_mild = df.groupby("wedstrijd_id").apply(
    lambda match_data: heeft_schepcorner_impact_mild(match_data["uitslag"].iloc[0], match_data)
)

# Voeg het resultaat toe aan het dataframe
df = df.merge(impact_resultaten_mild.rename("schepcorner_impact_mild"), on="wedstrijd_id")

# Print het resultaat
print(df[["wedstrijd_id", "schepcorner_impact_mild"]].drop_duplicates())


In [ ]:

# Voeg het resultaat toe aan het dataframe
df = df.merge(impact_resultaten_mild.rename("schepcorner_impact_mild"), on="wedstrijd_id")

# Print het resultaat
print(df[["wedstrijd_id", "schepcorner_impact_mild"]].drop_duplicates())
